In [1]:
import torch
import train_methods as tm
from tqdm import tqdm
from UNet import Unet
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# global var
C = 16                  # number of channels to give the model
batch_size = 16
epochs = 5
learning_rate = 1e-2
# act_batch_size = 1
momentum = 0.99
device = "cuda" if torch.cuda.is_available() else "cpu"
channels = [C, 64, 128, 256, 512, 1024]
w0 = 10
sigma = 5
model_path = './model.pt'
Train_image_dir = 'imagesTr'
Train_label_dir = 'labelsTr'
Val_image_dir = 'imagesVa'
Val_label_dir = 'labelsVa'

/home/or/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# Early stopping
es = tm.EarlyStopping(patience=100, fname=model_path)


# debug
imagesTR_len = len(os.listdir("/home/or/PycharmProjects/AMOS/imagesTr"))
print(f"Number of files in imagesTr: {imagesTR_len}")


# Make progress bars
pbar_epoch = tqdm(total=epochs, unit='epoch', position=0, leave=False)
pbar_train = tqdm(total=imagesTR_len, unit='batch', position=1, leave=False)

# Make model
model = Unet(channels=channels, no_classes=1).double().to(device)

# Make adam optimiser
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.001,  # Learning rate
    betas=(0.9, 0.999),  # Coefficients used for computing running averages of gradient and its square
    eps=1e-08,  # Term added to denominator to improve numerical stability
    weight_decay=0  # Weight decay (L2 penalty)
)

# Make loss
criterion = torch.nn.CrossEntropyLoss()

Number of files in imagesTr: 240


  0%|          | 0/240 [00:00<?, ?batch/s]

In [4]:
# Load checkpoint (if it exists)
cur_epoch = 0
if os.path.isfile(model_path):
    checkpoint = torch.load(model_path)
    cur_epoch = checkpoint['epoch']
    es.best_loss = checkpoint['loss']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Hold stats for training process
stats = {'epoch': [], 'train_loss': [], 'val_loss': []}

# Training  / validation loop
for epoch in range(cur_epoch, epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Train / validate
    pbar_epoch.set_description_str(f'Epoch {epoch + 1}')
    train_loss = tm.train(model, optimizer, batch_size, Train_image_dir, Train_label_dir, criterion, device)
    val_loss = tm.validate(model, batch_size, Val_image_dir,Val_label_dir, criterion, device)


    # Append stats
    stats['epoch'].append(epoch)
    stats['train_loss'].append(train_loss)
    stats['val_loss'].append(val_loss)

    # Early stopping (just saves model if validation loss decreases when: pass)
    if es(epoch, val_loss, optimizer, model): pass

    # Update progress bars
    pbar_epoch.set_postfix(train_loss=train_loss, val_loss=val_loss)
    pbar_epoch.update(1)
    pbar_train.reset()

Epoch 1:   0%|          | 0/5 [00:00<?, ?epoch/s]

Epoch 1/5


UnidentifiedImageError: cannot identify image file '/home/or/PycharmProjects/AMOS/imagesTr/amos_0142.nii.gz'

In [ ]:

# Define the directory containing .nii.gz files
directory_path = Train_image_dir  # Replace with the path to your directory

global_count = 0
count = 0
# Iterate over all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.nii.gz'):  # Check if the file is a .nii.gz file
        file_path = os.path.join(directory_path, filename)

        try:
            # Load the NIfTI file
            img = nib.load(file_path)
            data = img.get_fdata()

            # Print the filename and shape of the image
            if data.shape[0] == data.shape[1]:
                count += data.shape[2]
            print(f"File: {filename}, Shape: {data.shape}")
            global_count += data.shape[2]
        except Exception as e:
            print(f"Could not read file {filename}: {e}")

print(f"The amount of right pictures {count}")
print(f"the overall number of picture {global_count}")


In [ ]:
# Define directories
Train_image_dir = 'imagesTr'
Train_label_dir = 'labelsTr'

# Get list of file names in each directory
image_files = set(os.listdir(Train_image_dir))
label_files = set(os.listdir(Train_label_dir))

# Find files in imagesTr but not in labelsTr
extra_image_files = image_files - label_files

# Print the results
if extra_image_files:
    print("Files in 'imagesTr' but not in 'labelsTr':")
    for file in extra_image_files:
        print(file)
else:
    print("No extra files in 'imagesTr'.")


In [ ]:
# load model

model      = Unet(channels = channels, no_classes = 1).double().to(device)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Make loss
criterion = torch.CrossEntropyLoss()
with torch.no_grad():

    for  (X, y) in enumerate(test_loader):

        # Forward
        y_hat = model(X)
        y_hat = torch.sigmoid(y_hat)


        # Convert to numpy
        X = np.squeeze(X.cpu().numpy())
        y = np.squeeze(y.cpu().numpy())
        y_hat = np.squeeze(y_hat.detach().cpu().numpy())

        # Make mask
        y_hat2 = y_hat > 0.5

        # plot
        fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (8, 8))

        ax[0].imshow(y, 'gray', interpolation = None)
        ax[0].axis('off');
        ax[0].set_title('Target');

        ax[1].imshow(y_hat, 'gray', interpolation = None)
        ax[1].axis('off');
        ax[1].set_title('Prediction');